# NYC Rideshare Forecasting Pipeline - Part 2: Data Validation

**Author:** K Flowers  
**GitHub:** [github.com/KRFlowers](https://github.com/KRFlowers)  
**Date:** December 2025

This notebook validates the 684M NYC FHV (Uber/Lyft) trip records acquired during the download phase. It flags invalid records based on duration, distance, and fare thresholds. Finally, it outputs a clean dataset for EDA and modeling.


**Pipeline Position:** Notebook 2 of 4 -- Data Validation

- 00_data_download.ipynb
- 01_data_validation.ipynb ← **this notebook**
- 02_exploratory_analysis.ipynb
- 03_demand_forecasting.ipynb

**Objective:** Validate raw trip records against thresholds for duration, distance, and fare. Create a clean dataset containing only valid data. 

**Technical Approach:**
- Use DuckDB for memory-efficient processing of 18GB dataset
- Set validation thresholds for duration, distance, and fare fields
- Flag invalid records and save to a new dataset to preserve the original full dataset
- Apply validation across the full dataset. Zone-level temporal coverage is assessed in 02_exploratory_analysis.ipynb.

**Inputs:**
- `data/raw/combined_fhvhv_tripdata.parquet` -- Combined trip data (18GB)

**Outputs:**
- `data/validated/fhvhv_valid_data_for_eda.parquet` - Clean dataset for EDA (683M records)
- `data/validated/fhvhv_all_data_flagged.parquet` - All records with validation flags (684M)
- `data/quality_reports/validation_report.csv` - Validation metrics by rule

**Limitations:** Thresholds are based on practical reasoning, not statistically derived. Zero-fare trips are retained as valid.

**Runtime:** ~30 minutes (flagging ~20 min, validation counts ~10 min)

## 1. Configure Environment

### 1.1 Import Libraries

In [1]:
# Standard library
from pathlib import Path
import warnings

# Core data libraries
import pandas as pd
import duckdb
from datetime import datetime

### 1.2 Set Display and Plot Options

In [2]:
# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.4f}'.format)

### 1.3 Set Paths and Constants

In [3]:
# Project Constants
PROJECT_YEARS = [2022, 2023, 2024]
TLC_DATASET = 'fhvhv'

# Paths
PROJECT_ROOT = Path("..").resolve()
RAW_DIR = PROJECT_ROOT / "data" / "raw"
VALIDATED_DIR = PROJECT_ROOT / "data" / "validated"
REPORTS_DIR = PROJECT_ROOT / "data" / "quality_reports"

# Input/Output Files
INPUT_FILE = RAW_DIR / f"combined_{TLC_DATASET}_tripdata.parquet"
FLAGGED_FILE = VALIDATED_DIR / f"{TLC_DATASET}_all_data_flagged.parquet"
EDA_FILE = VALIDATED_DIR / f"{TLC_DATASET}_valid_data_for_eda.parquet"

# Create Directories
VALIDATED_DIR.mkdir(parents=True, exist_ok=True)
REPORTS_DIR.mkdir(parents=True, exist_ok=True)

print(f"Config loaded: {INPUT_FILE.name}")

Config loaded: combined_fhvhv_tripdata.parquet


### 1.4 Create Database Connection
Initialize DuckDB connection and configure performance settings. Database connection is required to validate the combined trip data file.

In [4]:
# Initialize DuckDB connection with optimized settings
con = duckdb.connect()
con.execute("SET threads=4")
con.execute("SET preserve_insertion_order=false")
#con.execute("SET enable_progress_bar = true")
#con.execute("SET progress_bar_time = 2000")
print("DuckDB connection established")

DuckDB connection established


## 2. Review Data
Reviews the dataset structure, date range coverage, and missing values.

### 2.1 Review Columns

In [5]:
# Display total record count
total_records = con.execute(f"SELECT COUNT(*) FROM '{INPUT_FILE}'").fetchone()[0]
print(f"Total records: {total_records:,}\n")

# Review column names and data types
con.execute(f"DESCRIBE SELECT * FROM '{INPUT_FILE}'").df()[['column_name', 'column_type']]

Total records: 684,376,551



,column_name,column_type
0,hvfhs_license_num,VARCHAR
1,dispatching_base_num,VARCHAR
2,originating_base_num,VARCHAR
3,request_datetime,TIMESTAMP
4,on_scene_datetime,TIMESTAMP
5,pickup_datetime,TIMESTAMP
6,dropoff_datetime,TIMESTAMP
7,PULocationID,BIGINT
8,DOLocationID,BIGINT
9,trip_miles,DOUBLE


### 2.2 Check Date Range

In [6]:
# Check date range of pickup_datetime to verify coverage period
date_range = con.execute(f"""
    SELECT 
        MIN(pickup_datetime) as earliest,
        MAX(pickup_datetime) as latest
    FROM '{INPUT_FILE}'
""").df()

print(f"Date range: {date_range['earliest'].iloc[0]} to {date_range['latest'].iloc[0]}")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Date range: 2022-01-01 00:00:00 to 2024-12-31 23:59:59


### 2.3 Check Missing Values
Identify null values by column. High-null columns will be excluded during aggregation in EDA, not removed here.

In [7]:
# Get list of column names
columns = con.execute(f"DESCRIBE SELECT * FROM '{INPUT_FILE}'").df()['column_name'].tolist()

# Build SQL to count NULLs per column using CASE WHEN - single scan counts nulls for all columns once
null_count_sql = f"""
    SELECT 
        {', '.join([f"SUM(CASE WHEN {col} IS NULL THEN 1 ELSE 0 END) AS {col}_null_count" for col in columns])}
    FROM '{INPUT_FILE}'
"""

# Execute query and get results as tuple
null_counts = con.execute(null_count_sql).fetchone()

# Calculate null percentages
null_pct = [(count / total_records) * 100 for count in null_counts]

# Display results 
print(f"{'Column':<25} {'Null Count':<12} {'Null Percentage'}")
print("-" * 50)
for col, count, pct in zip(columns, null_counts, null_pct):
    print(f"{col:<25} {count:<12} {pct:>15.2f}%")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Column                    Null Count   Null Percentage
--------------------------------------------------
hvfhs_license_num         0                       0.00%
dispatching_base_num      0                       0.00%
originating_base_num      183954837              26.88%
request_datetime          0                       0.00%
on_scene_datetime         183891654              26.87%
pickup_datetime           0                       0.00%
dropoff_datetime          0                       0.00%
PULocationID              0                       0.00%
DOLocationID              0                       0.00%
trip_miles                0                       0.00%
trip_time                 0                       0.00%
base_passenger_fare       0                       0.00%
tolls                     0                       0.00%
bcf                       0                       0.00%
sales_tax                 0                       0.00%
congestion_surcharge      0                       0.00

**Interpretation:** Core fields that will be used for analysis have zero nulls:
- `pickup_datetime`
- `PULocationID`
- `trip_time`
- `trip_miles`
- `base_passenger_fare`

The two high-null columns (`originating_base_num`, `on_scene_datetime`) are not used in this analysis.

**Section 2 Summary:**
- Dataset contains 684M records across 24 columns spanning 2022-01-01 to 2024-12-31
- All core analysis fields (pickup_datetime, PULocationID, trip_time, trip_miles, base_passenger_fare) have zero nulls
- Two high-null columns (originating_base_num, on_scene_datetime at ~27%) are not needed for demand analysis
- No structural issues detected; data is ready for threshold-based validation

## 3. Validate Data
Flags all records based on duration, distance, and fare thresholds. Creates two output datasets: one with all 684M records and quality flags, and one with only valid records for analysis.

### 3.1 Set Validation Thresholds

In [8]:
# Set validation thresholds for key fields that impact analysis

DURATION_MIN = 60          # 1 min - filters GPS errors, keeps short trips
DURATION_MAX = 43200       # 12 hrs - covers NYC to Philadelphia
DURATION_EXTREME = 604800  # 7 days - obvious corruption
DISTANCE_MIN = 0.1         # Filters GPS noise
DISTANCE_MAX = 200         # NYC-Philadelphia service area
FARE_MIN = 0               # No negative fares ($0 allowed for promos)
FARE_MAX = 500             # 99.9th percentile ~$150, allows surge

### 3.2 Flag Records Against Thresholds 

In [9]:
print("Creating flagged dataset...")
print("This will take approximately 10 minutes for 684M records\n")

# Validate key fields, create flags and save dataset with flags
con.execute(f"""
    COPY (
        SELECT 
            *,
            
            -- DURATION FLAGS - Checking null, zero/negative, min, max, and extreme
            (trip_time IS NULL) AS flag_duration_null,
            (trip_time <= 0) AS flag_duration_zero_negative,
            (trip_time < {DURATION_MIN}) AS flag_duration_too_short,
            (trip_time > {DURATION_MAX}) AS flag_duration_exceeds_max,
            (trip_time > {DURATION_EXTREME}) AS flag_duration_extreme,
            
            -- DISTANCE FLAGS - Checking null, negative, min, and max 
            (trip_miles IS NULL) AS flag_distance_null,
            (trip_miles < 0) AS flag_distance_negative,
            (trip_miles < {DISTANCE_MIN}) AS flag_distance_too_short,
            (trip_miles > {DISTANCE_MAX}) AS flag_distance_exceeds_max,
            
            -- FARE FLAGS - Checking null, negative, zero, and extreme high 
            (base_passenger_fare IS NULL) AS flag_fare_null,
            (base_passenger_fare < {FARE_MIN}) AS flag_fare_negative,
            (base_passenger_fare = 0) AS flag_fare_zero,
            (base_passenger_fare > {FARE_MAX}) AS flag_fare_extreme_high,
            
            -- APPLY MASTER VALIDITY FLAG - Record is valid if all checks pass (zero fare is allowed)
            (
                trip_time IS NOT NULL AND
                trip_time >= {DURATION_MIN} AND 
                trip_time <= {DURATION_MAX} AND
                trip_miles IS NOT NULL AND
                trip_miles >= {DISTANCE_MIN} AND
                trip_miles <= {DISTANCE_MAX} AND
                base_passenger_fare IS NOT NULL AND
                base_passenger_fare >= {FARE_MIN} AND
                base_passenger_fare <= {FARE_MAX}
            ) AS is_valid
            
        FROM '{INPUT_FILE}'
    ) TO '{FLAGGED_FILE}' (FORMAT PARQUET)
""")

# Verify the flagged dataset was created successfully
flagged_count = con.execute(f"SELECT COUNT(*) FROM '{FLAGGED_FILE}'").fetchone()[0]
print(f"Flagged dataset created: {flagged_count:,} records")

Creating flagged dataset...
This will take approximately 10 minutes for 684M records



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Flagged dataset created: 684,376,551 records


In [10]:
# Review a sample of flagged records to verify validation logic
con.execute(f"""
    SELECT 
        trip_time,
        trip_miles,
        base_passenger_fare,
        flag_duration_too_short,
        flag_distance_too_short,
        flag_fare_negative,
        is_valid
    FROM '{FLAGGED_FILE}'
    WHERE is_valid = FALSE
    LIMIT 20
""").df()

,trip_time,trip_miles,base_passenger_fare,flag_duration_too_short,flag_distance_too_short,flag_fare_negative,is_valid
0,1524,13.0900,-11.3500,False,False,True,False
1,7723,1.1900,-2.1200,False,False,True,False
2,813,4.1600,-2.7500,False,False,True,False
3,1371,8.2100,-21.3800,False,False,True,False
4,1913,18.8100,-12.8800,False,False,True,False
5,1416,8.0500,-3.2800,False,False,True,False
6,1845,11.3600,-11.2600,False,False,True,False
7,73,0.0250,13.4700,False,True,False,False
8,687,6.0800,-8.3500,False,False,True,False
9,184,0.0100,10.7900,False,True,False,False


### 3.3 Count Flagged Records
Counts invalid records overall and then by zone.  Verify no zone exceeds the 1% exclusion threshold.

In [11]:
# Count all validation flags in one query
validation_stats = con.execute(f"""
    SELECT 
        COUNT(*) as total,
        
        -- Duration issues
        SUM(CAST(flag_duration_null AS INTEGER)) as dur_null,
        SUM(CAST(flag_duration_zero_negative AS INTEGER)) as dur_zero_neg,
        SUM(CAST(flag_duration_too_short AS INTEGER)) as dur_too_short,
        SUM(CAST(flag_duration_exceeds_max AS INTEGER)) as dur_exceeds_max,
        SUM(CAST(flag_duration_extreme AS INTEGER)) as dur_extreme,
        
        -- Distance issues
        SUM(CAST(flag_distance_null AS INTEGER)) as dist_null,
        SUM(CAST(flag_distance_negative AS INTEGER)) as dist_negative,
        SUM(CAST(flag_distance_too_short AS INTEGER)) as dist_too_short,
        SUM(CAST(flag_distance_exceeds_max AS INTEGER)) as dist_exceeds_max,
        
        -- Fare issues
        SUM(CAST(flag_fare_null AS INTEGER)) as fare_null,
        SUM(CAST(flag_fare_negative AS INTEGER)) as fare_negative,
        SUM(CAST(flag_fare_zero AS INTEGER)) as fare_zero,
        SUM(CAST(flag_fare_extreme_high AS INTEGER)) as fare_extreme_high,
        
        -- Overall validity
        SUM(CAST(is_valid AS INTEGER)) as valid,
        SUM(CAST(NOT is_valid AS INTEGER)) as invalid
        
    FROM '{FLAGGED_FILE}'
""").fetchone()

# Unpack results
(total, 
 dur_null, dur_zero_neg, dur_too_short, dur_exceeds_max, dur_extreme,
 dist_null, dist_negative, dist_too_short, dist_exceeds_max,
 fare_null, fare_negative, fare_zero, fare_extreme_high,
 valid, invalid) = validation_stats

print(f"Validation counts complete: {total:,} records")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Validation counts complete: 684,376,551 records


In [12]:
# Count each validation flag
flag_counts_df = con.execute(f"""
    SELECT 
        SUM(CAST(flag_duration_null AS INT)) as flag_duration_null,
        SUM(CAST(flag_duration_zero_negative AS INT)) as flag_duration_zero_negative,
        SUM(CAST(flag_duration_too_short AS INT)) as flag_duration_too_short,
        SUM(CAST(flag_duration_exceeds_max AS INT)) as flag_duration_exceeds_max,
        SUM(CAST(flag_duration_extreme AS INT)) as flag_duration_extreme,
        SUM(CAST(flag_distance_null AS INT)) as flag_distance_null,
        SUM(CAST(flag_distance_negative AS INT)) as flag_distance_negative,
        SUM(CAST(flag_distance_too_short AS INT)) as flag_distance_too_short,
        SUM(CAST(flag_distance_exceeds_max AS INT)) as flag_distance_exceeds_max,
        SUM(CAST(flag_fare_null AS INT)) as flag_fare_null,
        SUM(CAST(flag_fare_negative AS INT)) as flag_fare_negative,
        SUM(CAST(flag_fare_zero AS INT)) as flag_fare_zero,
        SUM(CAST(flag_fare_extreme_high AS INT)) as flag_fare_extreme_high
    FROM '{FLAGGED_FILE}'
""").df().T

flag_counts_df.columns = ['count']
flag_counts_df['pct'] = (flag_counts_df['count'] / total * 100).apply(lambda x: f"{x:.3f}%")
flag_counts_df['count'] = flag_counts_df['count'].apply(lambda x: f"{x:,.0f}")

print(f"Total: {total:,} | Valid: {valid:,} ({valid/total*100:.2f}%) | Invalid: {invalid:,} ({invalid/total*100:.2f}%)\n")
flag_counts_df

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Total: 684,376,551 | Valid: 683,780,462 (99.91%) | Invalid: 596,089 (0.09%)



,count,pct
flag_duration_null,0,0.000%
flag_duration_zero_negative,65,0.000%
flag_duration_too_short,"60,163",0.009%
flag_duration_exceeds_max,57,0.000%
flag_duration_extreme,0,0.000%
flag_distance_null,0,0.000%
flag_distance_negative,0,0.000%
flag_distance_too_short,"251,660",0.037%
flag_distance_exceeds_max,"4,411",0.001%
flag_fare_null,0,0.000%


In [13]:
# Verify validation counts are consistent
assert valid + invalid == total, f"Count mismatch: {valid} + {invalid} != {total}"
assert valid > 0, "No valid records found"
assert valid / total > 0.95, f"Unexpectedly low valid rate: {valid/total:.2%}"

print(f"Validation verified: {valid:,} + {invalid:,} = {total:,}")

Validation verified: 683,780,462 + 596,089 = 684,376,551


**Interpretation:** 99.91% of records passed validation (683.8M of 684.4M). Only 596K excluded. Thresholds were set to catch errors without removing legitimate edge cases.

In [14]:
# Check invalid rate by zone to verify no bias by location
zone_validity_df = con.execute(f"""
    SELECT 
        PULocationID as zone_id,
        COUNT(*) as total,
        SUM(CASE WHEN is_valid = FALSE THEN 1 ELSE 0 END) as invalid,
        ROUND(SUM(CASE WHEN is_valid = FALSE THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as invalid_pct
    FROM '{FLAGGED_FILE}'
    GROUP BY PULocationID
    HAVING invalid_pct > 1.0
    ORDER BY invalid_pct DESC
""").df()

if len(zone_validity_df) == 0:
    print("No zones exceed 1% invalid rate")
else:
    print(f"{len(zone_validity_df)} zones exceed 1% invalid rate:")
    print(zone_validity_df)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

1 zones exceed 1% invalid rate:
   zone_id  total  invalid  invalid_pct
0        1     67  15.0000      22.3900


**Section 3 Summary:**
- 99.91% of records passed all validation checks (683.8M of 684.4M retained)
- No individual field exceeded a 0.1% invalid rate
- Negative fares were the most common flag (320K records, 0.047%)
- Only Zone 1 exceeded 1% invalid rate due to low trip volume (67 trips); no systemic zone bias detected

**Interpretation:** Zone 1 exceeds 1% invalid rate (22%) but has only 67 total trips. This zone is filtered out in EDA due to incomplete temporal coverage.

### 3.4 Save Valid Records Dataset
(~10 min)

In [15]:
# Create EDA dataset with only valid records, excluding flag columns
con.execute(f"""
    COPY (
        SELECT * EXCLUDE (
            flag_duration_null, 
            flag_duration_zero_negative, 
            flag_duration_too_short, 
            flag_duration_exceeds_max, 
            flag_duration_extreme,
            flag_distance_null, 
            flag_distance_negative, 
            flag_distance_too_short, 
            flag_distance_exceeds_max,
            flag_fare_null, 
            flag_fare_negative, 
            flag_fare_zero, 
            flag_fare_extreme_high,
            is_valid
        )
        FROM '{FLAGGED_FILE}'
        WHERE is_valid = true
    ) TO '{EDA_FILE}' (FORMAT PARQUET)
""")
print(f"EDA dataset saved: {EDA_FILE.name} ({valid:,} records)\n")

# Preview saved dataset
con.execute(f"SELECT * FROM '{EDA_FILE}' LIMIT 3").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

EDA dataset saved: fhvhv_valid_data_for_eda.parquet (683,780,462 records)



,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,trip_time,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
0,HV0003,B03404,B03404,2022-01-01 00:05:31,2022-01-01 00:05:40,2022-01-01 00:07:24,2022-01-01 00:18:28,170,161,1.1800,664,24.9000,0.0000,0.7500,2.2100,2.7500,0.0000,0.0000,23.0300,N,N,,N,N
1,HV0003,B03404,B03404,2022-01-01 00:19:27,2022-01-01 00:22:08,2022-01-01 00:22:32,2022-01-01 00:30:12,237,161,0.8200,460,11.9700,0.0000,0.3600,1.0600,2.7500,0.0000,0.0000,12.3200,N,N,,N,N
2,HV0003,B03404,B03404,2022-01-01 00:43:53,2022-01-01 00:57:37,2022-01-01 00:57:37,2022-01-01 01:07:32,237,161,1.1800,595,29.8200,0.0000,0.8900,2.6500,2.7500,0.0000,0.0000,23.3000,N,N,,N,N


### 3.5 Save Flag Counts Dataset

In [16]:
# Save validation flag counts to CSV
flag_counts_df.to_csv(REPORTS_DIR / "validation_report.csv")
print(f"Saved: validation_report.csv\n")

# Preview saved report
flag_counts_df

Saved: validation_report.csv



,count,pct
flag_duration_null,0,0.000%
flag_duration_zero_negative,65,0.000%
flag_duration_too_short,"60,163",0.009%
flag_duration_exceeds_max,57,0.000%
flag_duration_extreme,0,0.000%
flag_distance_null,0,0.000%
flag_distance_negative,0,0.000%
flag_distance_too_short,"251,660",0.037%
flag_distance_exceeds_max,"4,411",0.001%
flag_fare_null,0,0.000%


In [17]:
# Close DuckDB connection
con.close()
print("Pipeline complete")

Pipeline complete


## References

- NYC Taxi and Limousine Commission. (2025). TLC Trip Record Data. https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

## 4. Validation Summary

Total: 684,376,551 | Valid: 683,780,462 (99.91%) | Invalid: 596,089 (0.09%)

- Approximately **99.91%** of total trip records passed validation checks and were retained for analysis
- No field exceeded .1% invalid rate
- A small number of zones exhibited higher invalid percentages due to extremely low trip counts
- The validated dataset preserves sufficient volume and coverage for reliable zone-level demand analysis


## Conclusion

**Validation Result:**

The raw dataset is clean and ready for analysis. 99.91% of records (683.8M
of 684.4M) passed validation checks, with no field exceeding a 0.1%
invalid rate.

**Key Findings:**
- **Overall quality:** 99.91% of records passed validation (596K excluded)
- **Duration flags:** 60K trips under 1 minute, 57 trips over 12 hours
- **Distance flags:** 252K trips under 0.1 miles, 4.4K over 200 miles
- **Fare flags:** 320K negative fares, 13.5K extreme fares over $500
- **Zone bias:** Only 1 zone exceeded 1% invalid rate (Zone 1, 67 total trips)

**Validation Approach:**
- Reviewed dataset structure, date range, and missing values
- Set duration, distance, and fare thresholds
- Flagged all 684M records against thresholds
- Verified no zone-level bias in exclusion rates
- Saved flagged dataset and clean dataset separately

**Limitations:**
- Thresholds are based on practical reasoning, not statistically derived
- No statistical outlier removal was applied. Aggregation to daily trip counts in EDA minimizes their impact on the forecasting target.
- No cross-field validation (fare vs distance consistency)

**Outputs:**
- `data/validated/fhvhv_valid_data_for_eda.parquet` -- Clean dataset (683M records)
- `data/validated/fhvhv_all_data_flagged.parquet` -- All records with validation flags (684M)
- `data/quality_reports/validation_report.csv` -- Validation metrics by rule

**Next Steps:**
Proceed to **02_exploratory_analysis.ipynb** 


## Conclusion
**EDA Result:**


EDA discovered demand characteristics (stable weekly patterns, stationary variance, and
high inter-zone correlation) that favor lag-based models over trend-focused
approaches. The
following models will be evaluated in Notebook 03:
- **Seasonal Naive:** Baseline using 7-day lag to reflect dominant weekly cycle
- **Prophet:** Tests whether trend/seasonality decomposition adds value over simpler methods
- **XGBoost:** Leverages multiple lag features to capture the stable weekly structure

EDA also identified a strong zone-to-global correlation pattern in 195 zones, indicating a
shared weekly demand structure. These zones represent 82% of total trips. Based on this consistency, a single modeling approach will be applied across all zones. The remaining 61 low-correlation zones (airports, entertainment districts) have distinct demand drivers and were saved separately for future evaluation.

**Key Findings:**
- **Stationarity:** Demand is stationary with constant variance, no detrending needed
- **Weekly dominance:** Weekend demand +13% vs weekday; weekly cycle is the primary pattern
- **Zone stability:** Within-zone demand is consistent (median CV 0.17)
- **Correlation structure:** 76% of zones (82% of trips) correlate strongly with global patterns
- **Segmentation:** 61 low-correlation zones (airports, entertainment districts) require separate treatment

**EDA Approach:**
- Aggregated 683M trip records to daily counts by zone
- Tested stationarity with Augmented Dickey-Fuller test
- Assessed variance stability with 30-day rolling statistics
- Analyzed weekly, monthly, and seasonal patterns
- Measured zone-to-global correlation to segment zones

**Outputs:**
- `zone_daily_full.parquet` -- Complete dataset (256 zones)
- `zone_daily_high_corr.parquet` -- High-correlation zones (195 zones)
- `zone_daily_low_corr.parquet` -- Low-correlation zones (61 zones)

**Next Steps:**
Proceed to **03_demand_forecasting.ipynb** to build and evaluate
forecasting models on the high-correlation zone subset.
